# Word2vec y redes neuronales para modelos de lenguaje



## Modelos de lenguaje


(Referencia: [Bengio et al](http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf))


Si $w=w_1w_2\cdots w_N$ es una frase, y las $w$ representan palabras,  un **modelo de lenguaje**
es una distribución de probabilidad sobre posibles oraciones

$$P(w_1w_2\cdots w_N).$$

Estos modelos son importantes en varias áreas de procesamiento de lenguaje natural. Por ejemplo, son utilizados en reconocimiento de lenguaje hablado, reconocimiento de escritura, corrección de ortografía, etc. 

Por ejemplo, en corrección de ortografía (ejemplo simple), el modelo de lenguaje es el que nos permite escoger entre varias alternativas de corrección: si escribimos *'el peroo corre'*, tenemos como alternativas *'el pero corre'*, *'el perno corre'* o *'el perro corre'*. Un modelo de lenguaje adecuado señalaría como corrección apropiada *'el perro corre'*, pues es una oración mucho más probable que las otras dos.


Muchas veces los modelos de lenguaje se construyen a partir de probabilidades condicionales:

$$P(w|w_1\ldots w_s),$$

para cualquier palabra $w$ y frase $w_1\ldots w_s$. Podemos utilizar entonces la regla del producto para calcular cualquier $P(w_1w_2\cdots w_N).$ En la práctica, sin embargo, es difícil construir estas probabiilidades para cualquier frase $w_1\ldots w_s$ (hay una cantidad inmensa de posibles frases a las qué condicionar), y aproximamos utilizando
una dependencia relativamente corta. Por ejemplo, el modelo de trigramas está dado por las probabilidades

$$P(w_3 | w_1w_2),$$

y usamos estas para calcular la probabilidad de una frase que puede ser de longitud arbitraria $N$ usando la regla del producto.

Con una colección de textos grandes,  se pueden obtener buenos modelos simplemente contando para estimar estas probabilidades ([Google ngram viewer](https://books.google.com/ngrams)). 




## Redes neuronales y modelos de lenguaje


### Idea básica
Estos modelos de lenguaje también pueden construirse con redes neuronales. La idea básica es
que la capa de entrada sea por ejemplo el par de palabras $w_1, w_2$, codificadas con vectores indicadores.

Por ejemplo, si el lenguaje tiene $V=5$ palabras, que son 'hola','adiós','y','el','día', identificamos cada palabra con un entero 1,2,3,4,5. Si observamos  el par de palabras 'hola y', codificaríamos (codificación dummy o indicadoras) 'hola' a (1,0,0,0,0) y 'y' a (0,0,1,0,0), y la capa de entrada (con 10 variables) sería entonces
$$z=((1,0,0,0,0),(0,0,1,0,0)).$$

Después de esta capa pondremos algunas capas ocultas, y finalmente tenemos la capa de salida
es un vector de tamaño $V=5$.
$$(p_1,\ldots,p_5)$$
que nos da la probabilidad de que cada posible palabra ocurra después de 'hola y'.


### Arquitectura

La primera idea importante para estas redes está en la arquitectura de la primera capa, que está definida por una matrix $C$ de tamaño $m\times V$. 

Obsérvese que si $e_i$ es un vector indicador ($i$-ésima componente igual a 1 y el resto 0), $Ce$ es simplemente la columna $i$ de la matriz $C$. **En la primera capa, usamos la matriz de pesos $C$ para todas las palabras independientemente de su posición**.


En nuestro ejemplo, si $C$ de tamaño $m\times V=m\times 5$, haríamos

$$z=((1,0,0,0,0),(0,0,1,0,0)) = (e_1,e_3) = (Ce_1, Ce_3) = (x_1,x_2) = x,$$

donde nótese que usamos la misma matriz $C$ para ambos vectores. Tenemos entonces pesos compartidos de $C$ sobre las palabras de entrada. 

En esta primera capa, que sólo extrae los coeficientes apropiados, no hace falta agregar una función no-lineal (esta capa simplemente asigna un vector a cada palabra de entrada).


En el modelo de Bengio, seguimos con una capa oculta totalmente conexa y la capa de salida totalmente conexa:

La siguiente capa oculta es entonces

$$w = h( b+Hx),$$

donde $h$ es la función logística o tangente hiperbólica, aplicada componente a componente, y finalmente la capa de salida dada por

$$y = a+Uw$$
seguido de softmax
$$p_i = \frac{e^{y_i}}{\sum_j e^{y_j}}$$








### Resumen: modelo de red neuronal

Podemos entonces construir una red neuronal con 2 capas ocultas como sigue: Usemos el ejemplo de trigramas:

1. En la primera capa oculta, tenemos un mapeo de las entradas $w_1,\ldots, w_{n-1}$ a $x=C(w_1),\ldots, C(w_{n-1})$, donde $C$ es una función que mapea palabras a vectores de dimensión $d$. $C$ también se puede pensar como una matriz de dimensión $|V|$ por $d$. En la capa de entrada,

$$w_{n-2},w_{n-1} \to x = (C(w_{n-2}), C(w_{n-1})).$$

que es un vector de tamaño $2d$.


2. En la siguiente capa oculta tenemos una matriz de pesos $H$ y la función logística (o tangente hiperbólica) $\sigma$, como en una red neuronal usual. En esta capa calculamos
$$z = h (a + Hx),$$
que resulta en un vector de tamaño $h$. 

3. Finalmente, la capa de salida debe ser un vector de probabilidades
sobre todo el vocabulario $|V|$. En esta capa tenemos pesos $U$ y hacemos
$$y = b + Uh (z),$$
y finalmente usamos softmax para tener probabilidades que suman uno:
$$p_i = \frac{\exp (y_i) }{\sum_j exp(y_j)}.$$

En el ajuste maximizamos la verosimilitud:

$$\sum_t \log \hat{P}(w_{t,n}|w_{t,n-2}w_{t-n-1})$$


La representación en la referencia (Bagio) es:

![Imagen](1_neural_model.png)

Esta idea original ha sido explotada con éxito, aunque sigue siendo
muy intensivo en cómputo ajustar un modelo como este. Nótese que
el número de parámetros es del orden de $|V|(m+h)$, donde $|V|$ es el tamaño del vocabulario (decenas o cientos de miles),  $m$ es el tamaño de la representacion (cientos o miles) y $h$ es el número de nodos en la segunda capa (también cientos o miles).  Esto resulta en el mejor de los casos en modelos con miles de millones de parámetros. Adicionalmente, hay algunos cálculos costosos, como el softmax (donde hay que hacer una suma sobre el vocabulario completo). En el paper original se propone gradiente estocástico paralelizado.


### Representación de palabras

Un aspecto interesante de el modelo de arriba es que
nos da una representación vectorial de las palabras, en la forma
de los parámetros ajustados de la matriz $C$. Esta se puede entender
como una descripción numérica de cómo funciona una palabra en el contexto de su n-grama.

Por ejemplo, deberíamos encontrar que palabras como "perro" y "gato" tienen representaciones similares. La razón es que cuando aparecen,
las probabilidades sobre las palabras siguientes deberían ser similares, pues estas son dos palabras que se pueden usar en muchos contextos
compartidos.

También podríamos encontrar que palabras como perro, gato, águila, león, etc. tienen partes o entradas similares en sus vectores de representación, que es la parte que hace que funcionen como "animal mamífero" dentro de frases. 

Veremos que hay más razones por las que es interesante esta representación.


## Modelos de word2vec de Google

Referencias: [word2vec](http://arxiv.org/pdf/1301.3781.pdf)

Si lo que principalmente nos interesa es obtener la representación
vectorial de palabras, recientemente se descubrió que es posible 
simplificar considerablemente el modelo de arriba para poder entrenarlo mucho más rápido, y obtener una representación que en muchas tareas se desempeña muy bien.

Hay dos ideas básicas para reducir la complejidad del entrenamiento:

- Eliminar la segunda capa oculta.
- Cambiar la función objetivo (minimizar devianza/maximizar verosimilitud) por una más simple, mediante un truco que se llama "negative sampling".

Se usa un enfoque un poco distinto (no es necesario predecir la siguiente palabra) :  **intentamos predecir la palabra
central a partir de las que están alrededor**. Esto es porque para los problemas que se ha aplicado esta resulta una mejor estrategia (problema de analogías, por ejemplo, que veremos más adelante).

### Arquitectura continuous bag-of-words

La entrada es igual que en el modelo completo. En primer lugar,
simplificamos la segunda capa oculta pondiendo en $z$ el promedio de
los vectores $C(w_{n+1}), C(w_{n-1})$ (que rodean a $w_n$).  La última capa la dejamos igual, y por el momento:

![Imagen](cbow_fig.png)



El modelo se llama bag-of-words porque todas las entradas de la primera capa oculta contribuyen de la misma manera en la salida, independientemente del orden. Aunque esto no suena como buena idea para construir un modelo de lenguaje, veremos que resulta en una representación adecuada para algunos problemas.




1. En la primera capa oculta, tenemos un mapeo de las entradas $w_1,\ldots, w_{n-1}$ a $x=(C(w_1),\ldots, C(w_{n-1}))$, donde $C$ es una función que mapea palabras a vectores de dimensión $d$. $C$ también se puede pensar como una matriz de dimensión $|V|$ por $d$. En la capa de entrada,

$$w_{n+1},w_{n-1} \to x = (C(w_{n+1}), C(w_{n-1})).$$


2. En la siguiente "capa"" oculta simplemente sumamos las entradas de $x$ para obtener $z$, que es también un vector de longitud $d$. Aquí nótese que realmente no hay parámetros, solo una proyección.

3. Finalmente, la capa de salida debe ser un vector de probabilidades
sobre todo el vocabulario $|V|$. En esta capa tenemos pesos $U$ y hacemos ($h$ es la función logística):
$$y = b + Uh (z),$$
y finalmente usamos softmax para tener probabilidades que suman uno:
$$p_i = \frac{\exp (y_i) }{\sum_j exp(y_j)}.$$

En el ajuste maximizamos la verosimilitud:

$$\sum_t \log \hat{P}(w_{t,n}|w_{t,n+1} \cdots w_{t-n-1})$$

Todavía se propone una simplificación adicional

### Muestreo negativo

La siguiente simplificación consiste en cambiar la función objetivo. En word2vec puede usarse "muestreo negativo".

Para empezar, la función objetivo original (para un solo trigrama) es


$$E` = -\log \hat{P}(w_n|w_{n-1}w_{n+1}) = -y_w + \log\sum_j \exp(y_j).$$

La dificultad está en el segundo término, que es sobre todo el vocabulario en incluye todos los parámetros del modelo.


La idea del muestreo negativo es que si observamos
$w_n$ entre de $w_{n+1}$ y $w_{n-1}$, tomamos una muestra de $k$ palabras
$v_1,\ldots v_k$ al azar
(2-100, dependiendo del tamaño de la colección), y creamos $k$
"trigramas falsos" $w_{n-2} v_j w_{n-1}$, $j=1\ldots,k$. Minimizamos
en lugar de la observación de arriba

$$E = -\log h(y_w) - \sum_k \log h (-y_k).$$

Es decir, solo buscamos optimizar parámetros para separar lo mejor
que podamos la observación "verdadera" de las $k$ observaciones "falsas".



### Arquitectura continuous skip-gram


Es también posibles (y en algunos casos con mejores resultados) usar la siguiente estructura, donde intentamos predecir el contexto a partir de cada palabra:

![Imagen](skip-gram.png)




## Ejemplo

Consideramos una colección de fragmentos de noticias de periódicos españoles.

In [91]:
import pandas as pd

train_esp = pd.read_csv("datos/Es_Newspapers.txt",
                        delimiter="\t", header=None, quoting=3, encoding = 'utf-8')

train_esp.columns = ['fragmento']


In [2]:
train_esp

,fragmento
0,"En este sentido, señala que «no podemos consen..."
1,"""Cuando acabe la experiencia con el Inter no m..."
2,"«Teniendo salud se aguanta todo», dice Nati Mi..."
3,'72 días': De Danilo &ScaronerbedÅ¾ija. Croaci...
4,La Administración norteamericana no ha dado aú...
5,Tras ensalzar la figura de Arenas como ex mini...
6,"El doctor del área de Prehistoria de la UMU, L..."
7,"La diputada provincial de Servicios Sociales, ..."
8,"En este punto, la corte acepta las explicacion..."
9,"En Santander, la cesión de los terrenos en man..."


Por ejemplo

In [100]:

print train_esp["fragmento"][309891]



Es por ello, que esta noche de verano será una de las más alternativas y originales de las que se ofrecen en la Feria de Julio en Valencia.


Necesitaremos una función para separar oraciones en palabras (eliminamos signos de puntuación, números). También convertimos por facilidad todo a minúsculas:

In [105]:
def texto_palabras(oracion):  
    #dejar solo caracteres alfanuméricos
    oracion_texto = re.sub(u"[^a-zA-Záéíóúñ]"," ", oracion)
    #separar las palabras después de ponerlas en minúsculas
    palabras = oracion_texto.lower().split()
    return(palabras)    
palabras = texto_palabras( train_esp["fragmento"][309891])
for p in palabras:
    print p

es
por
ello
que
esta
noche
de
verano
será
una
de
las
más
alternativas
y
originales
de
las
que
se
ofrecen
en
la
feria
de
julio
en
valencia


Vamos a dividir los textos en oraciones, y necesitamos que cada una oración esté representada como una lista de palabras. Para eso usamos la siguiente función:


In [110]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')

def texto_oraciones( texto, tokenizer ):
    oraciones_lista = tokenizer.tokenize(texto.strip())
    oraciones = []
    for oracion in oraciones_lista:        
        if len(oracion) > 0:
            oraciones.append( texto_palabras( oracion ))
    return oraciones


Por ejemplo:

In [114]:
oraciones = []
for texto in train_esp["fragmento"][:3]:
    oraciones += texto_oraciones(texto, tokenizer)
print "Primera oración"
print oraciones[0]
print "Segunda oración"
print oraciones[1]


Primera oración
[u'en', u'este', u'sentido', u'se\xf1ala', u'que', u'no', u'podemos', u'consentir', u'que', u'se', u'repita', u'el', u'malogrado', u'caso', u'del', u'centro', u'de', u'transportes', u'de', u'benavente', u'donde', u'la', u'falta', u'de', u'control', u'ha', u'supuesto', u'un', u'c\xfamulo', u'de', u'irregularidades', u'que', u'rozan', u'lo', u'delictivo']
Segunda oración
[u'cuando', u'acabe', u'la', u'experiencia', u'con', u'el', u'inter', u'no', u'me', u'quedar\xe9', u'en', u'italia', u'sino', u'que', u'espero', u'ir', u'a', u'espa\xf1a', u'porque', u'mi', u'objetivo', u'es', u'ganar', u'los', u't\xedtulos', u'de', u'los', u'tres', u'campeonatos', u'm\xe1s', u'competitivos', u'del', u'mundo', u'afirm\xf3', u'el', u'que', u'fuera', u'entrenador', u'del', u'barcelona', u'b', u'y', u'a\xf1adi\xf3', u'me', u'falta', u'liga']


In [115]:
oraciones = []
for texto in train_esp["fragmento"]:
    oraciones += texto_oraciones(texto, tokenizer)

print len(oraciones)

718544

In [54]:
## Este código sólo es necesario si queremos ver las salidas durante el
## proceso de ajuste

import logging
import logging.handlers

log = logging.getLogger()
f = logging.Formatter("%(asctime)s - %(module)s.   %(funcName)s - %(levelname)s - %(message)s")
fh = logging.handlers.TimedRotatingFileHandler('log.txt', 'W6')
fh.setFormatter(f)
log.addHandler(fh)
log.setLevel(logging.INFO)
log.info('Prueba de logging.')




INFO:root:Prueba de logging.


In [118]:
# Valores para parámetros
num_features = 150    # Dimensión de los vectores de palabras
min_word_count = 40   # Mínima frecuencia para incluir en el modelo                        
num_workers = 6       # Número de procesos
context = 6         # El contexto es el tamaño de n-gramas (10 es 5 al principio y 5 al final)                                                                                    
downsampling = 0  # Los autores recomiendan submuestrar palabras muy frecuentes (0.0001 y 0.01)
k_negative = 30       # Cuántas muestras negativas

In [119]:
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(oraciones, workers=num_workers, hs=0,
                          size=num_features, min_count = min_word_count,
                          window = context, sample = downsampling, negative = k_negative)

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 260339 words, keeping 27849 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 520837 words, keeping 40519 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 780387 words, keeping 49659 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 1037751 words, keeping 57292 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 1300719 words, keeping 63935 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 1560025 words, keeping 69439 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 1817263 words, keeping 74735 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #80000, processed 2077525 words, keep

Training model...


In [120]:
model['rey']

array([-0.15794076, -0.15631652, -0.36642364,  0.04203986, -0.11601214,
        0.25709817, -0.4137066 , -0.34088609, -0.30878621,  0.11356426,
       -0.35285285, -0.33517739, -0.12443205,  0.04128909, -0.10767747,
        0.48707077, -0.21437369,  0.30905855,  0.06170418,  0.21863939,
        0.06672436,  0.03327598, -0.26200852, -0.142635  , -0.31035939,
        0.12735093, -0.15826052, -0.05973155,  0.05695412, -0.10330881,
       -0.09728532, -0.03760261, -0.328592  , -0.17433362, -0.58762729,
        0.47216046, -0.37357807,  0.34607968, -0.71195924, -0.15372393,
       -0.30316225,  0.13135159, -0.06422084, -0.02370847, -0.12462158,
        0.06934674,  0.04028832, -0.02416475, -0.15595137, -0.04859674,
        0.21444683, -0.05047542, -0.06709774, -0.12388422,  0.09956151,
        0.78626192, -0.37894425, -0.29213643, -0.37914097, -0.01673727,
        0.46280301, -0.02709499,  0.27914631, -0.3597815 ,  0.25356546,
       -0.28640878, -0.22427273, -0.23685978, -0.23272741, -0.47

In [121]:
len(model['rey'])

150

In [122]:
vocab = model.index2word
len(vocab)


22192

In [123]:
vocab[0:10]



[u'de', u'la', u'el', u'que', u'en', u'y', u'a', u'los', u'del', u'se']

# Espacio de representación de palabras

Una vez que tenemos la representación de palabras podemos usarlas en otros modelos, o aplicarlos en distintas tareas. En la librería gensim hay tres buenos ejemplo: palabras similares (usando distancia euclideana en espacio de representaciones), analogías y encontrar la palabra distinta de un grupo.

### Palabras similares

Utiliza distancia euclideana en el espacio de representación distribuida:

In [124]:
model.most_similar('lluvia', topn = 4)

INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors


[(u'tormenta', 0.8314812183380127),
 (u'humedad', 0.8150405883789062),
 (u'fr\xedo', 0.8086050152778625),
 (u'precipitaci\xf3n', 0.7927411794662476)]

In [125]:
model.most_similar('gol', topn = 10)

[(u'saque', 0.8230658173561096),
 (u'penalti', 0.8180238604545593),
 (u'remate', 0.8097467422485352),
 (u'set', 0.8084077835083008),
 (u'break', 0.807267963886261),
 (u'empate', 0.8063345551490784),
 (u'cabezazo', 0.7945455312728882),
 (u'c\xf3rner', 0.7887912392616272),
 (u'minuto', 0.7871119379997253),
 (u'contragolpe', 0.7827543616294861)]

In [126]:
model.most_similar('bonito', topn = 10)

[(u'emocionante', 0.9121171832084656),
 (u'divertido', 0.8937031626701355),
 (u'maravilloso', 0.8936442732810974),
 (u'doloroso', 0.879575252532959),
 (u'arriesgado', 0.877044141292572),
 (u'inc\xf3modo', 0.8764861226081848),
 (u'c\xf3modo', 0.873559296131134),
 (u'vistoso', 0.8731124401092529),
 (u'exigente', 0.8686376810073853),
 (u'agradable', 0.8677032589912415)]

In [127]:
model.most_similar('feo', topn = 10)

[(u'gracioso', 0.9655297994613647),
 (u'estupendo', 0.9500259160995483),
 (u'arriesgado', 0.9461573958396912),
 (u't\xedmido', 0.9422381520271301),
 (u'exagerado', 0.9421201944351196),
 (u'afortunado', 0.9383031129837036),
 (u'aburrido', 0.9380552172660828),
 (u'raro', 0.9332619309425354),
 (u'tonto', 0.9319804310798645),
 (u'guapo', 0.9303505420684814)]

### Analogías: respuesta de preguntas

Los vectores de esta representación contienen información sorprendente, tanto de sintaxis como de significado. Por ejemplo, podemos intentar contestar preguntas como:

*hombre es a rey como mujer es a ...*. 


La idea es que en el espacio de representaciones, podemos hacer aritmética básica de conceptos: buscamos encontrar una palabra x tal que los dos vectores v(rey) - v(hombre) y v(mujer) - v(x) sean muy similares.

![Imagen](vspace_ops.png)

In [152]:
model.most_similar(positive=['mujer',u'abuelo'], negative=['hombre'], topn=5)


[(u'abuela', 0.8424128293991089),
 (u'hermana', 0.8109989166259766),
 (u'madre', 0.7994287610054016),
 (u'novia', 0.7889715433120728),
 (u'amiga', 0.7852054238319397)]

In [153]:
model.most_similar(positive=['mujer','hombres'], negative=['hombre'], topn=5)

[(u'mujeres', 0.6479780673980713),
 (u'varones', 0.6390057802200317),
 (u'ancianos', 0.6382261514663696),
 (u'maltratadores', 0.6336269974708557),
 (u'homosexuales', 0.6328884363174438)]

In [155]:
model.most_similar(positive=['mujer','rey'], negative=['hombre'], topn=15)

[(u'invitaci\xf3n', 0.6186299324035645),
 (u'boda', 0.6086856126785278),
 (u'princesa', 0.606860339641571),
 (u'infanta', 0.6054365038871765),
 (u'viuda', 0.6032512187957764),
 (u'homil\xeda', 0.6028061509132385),
 (u'resurrecci\xf3n', 0.6002622842788696),
 (u'comuni\xf3n', 0.598954975605011),
 (u'tumba', 0.5981884002685547),
 (u'majestad', 0.5978625416755676),
 (u'bendici\xf3n', 0.5952019095420837),
 (u'oraci\xf3n', 0.5945830941200256),
 (u'reina', 0.5942752957344055),
 (u'borb\xf3n', 0.5942150354385376),
 (u'confederaciones', 0.5939369797706604)]

In [156]:
model.most_similar(positive=['nieve','verano'], negative=['arena'], topn=5)

[(u'invierno', 0.6695204973220825),
 (u'oto\xf1o', 0.6542546153068542),
 (u'agosto', 0.6259781718254089),
 (u'enero', 0.6249174475669861),
 (u'septiembre', 0.6161805987358093)]

In [158]:
model.most_similar(positive=['francia','londres'], negative=['inglaterra'], topn=5)

[(u'par\xeds', 0.7726121544837952),
 (u'berl\xedn', 0.7514835596084595),
 (u'mosc\xfa', 0.7252907752990723),
 (u'tokio', 0.7204474210739136),
 (u'estocolmo', 0.7150971293449402)]

In [159]:
model.most_similar(positive=[u'días','semana'], negative=[u'día'], topn=20)

[(u'meses', 0.6087338924407959),
 (u'semanas', 0.5610102415084839),
 (u'a\xf1os', 0.4978870451450348),
 (u'd\xe9cada', 0.492570698261261),
 (u'ejercicios', 0.4863927662372589),
 (u'lustros', 0.4832686483860016),
 (u'mandatos', 0.47152382135391235),
 (u'encuentros', 0.47019052505493164),
 (u'trimestres', 0.4672251045703888),
 (u'siglos', 0.4663507044315338),
 (u'primavera', 0.460256427526474),
 (u'veranos', 0.45867007970809937),
 (u'pretemporada', 0.45202308893203735),
 (u'instantes', 0.44825083017349243),
 (u'arrestos', 0.4478601813316345),
 (u'preparativos', 0.44780829548835754),
 (u'legislatura', 0.4448522627353668),
 (u'temporadas', 0.44212284684181213),
 (u'periodos', 0.4336188733577728),
 (u'quince', 0.4292360246181488)]

In [177]:
model.most_similar(positive=['sol','noche'], negative=[u'día'], topn=5)

[(u'playa', 0.59772127866745),
 (u'sombra', 0.569498598575592),
 (u'cola', 0.5649012327194214),
 (u'desembocadura', 0.5639711618423462),
 (u'orilla', 0.5566952228546143)]

In [184]:
model.most_similar(positive=[u'comer',u'noche'], negative=[u'día'], topn=5)

[(u'dormir', 0.692653477191925),
 (u'pasear', 0.6907150149345398),
 (u'beber', 0.687917172908783),
 (u'cenar', 0.6622120141983032),
 (u'cocinar', 0.650323212146759)]

### Palabra que no pertenece

Si tenemos una colección de palabras "perro gato cuchara", ¿cuál es la que no pertenece? Podemos usar los vectores. En este caso, calculamos el centroide de los vectores de las palabras proporcionadas, calculamos el centroide, y escogemos la palabra más lejana del centroide

In [196]:
model.doesnt_match(u'comida cena desayuno perro'.split())

u'perro'

In [163]:
model.doesnt_match('gol juego presidente remate portero'.split())

'presidente'

In [164]:
model.doesnt_match('francia inglaterra rusia comida italia'.split())

'comida'

In [199]:
print model.doesnt_match(u'juan maría pedro fernando josé alberto'.split())

maría


In [166]:
model.doesnt_match('uno dos tres francia cuatro cinco seis siete ocho'.split())

'francia'

In [200]:
model.doesnt_match('invierno verano otoño cuchara'.split())

'invierno'

### Usar los vectores de palabras

Podemos hacer más cosas extrayendo la matriz de vectores (por ejemplo, usar como entradas para otra tarea de predicción): 

In [144]:
type(model.syn0)

numpy.ndarray

In [145]:
model.syn0.shape

(22192, 150)

### Referencias
- [Word2vec en ejercicio de Kaggle](https://www.kaggle.com/c/word2vec-nlp-tutorial)

- [word2vec en Spotify, aplicando a playlists  junto con factorización de matrices (34)](http://www.slideshare.net/AndySloane/machine-learning-spotify-madison-big-data-meetup)

- [Código original y algunas explicaciones](https://code.google.com/p/word2vec/)

- [Paper original con varios resultados](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

- [Presentación](http://www.coling-2014.org/COLING%202014%20Tutorial-fix%20-%20Tomas%20Mikolov.pdf)

- [Otra explicación corta](http://arxiv.org/pdf/1402.3722v1.pdf)

- [Documentación de word2vec](https://radimrehurek.com/gensim/models/word2vec.html)

